In [14]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard
import datetime
import math

In [15]:
# Utilize VGG16 for Feature Extraction
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

In [16]:
# Freeze convolutional layers
for layer in vgg_base.layers:
    layer.trainable = False

In [17]:
# Data Preprocessing and Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'newdata/training_set',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'  # Set class_mode to 'binary' for binary classification
)

validation_generator = test_datagen.flow_from_directory(
    'newdata/test_set',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'  # Set class_mode to 'binary' for binary classification
)

Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


In [18]:
# Calculate steps_per_epoch and validation_steps
steps_per_epoch = math.ceil(train_generator.samples / train_generator.batch_size)
validation_steps = math.ceil(validation_generator.samples / validation_generator.batch_size)

print("Steps per epoch:", steps_per_epoch)
print("Validation steps:", validation_steps)

Steps per epoch: 251
Validation steps: 64


In [19]:
# Build and Train the Classification Model
model = Sequential([
    vgg_base,
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [20]:
# Model Compilation and Training
model.compile(optimizer=SGD(learning_rate=1e-4, momentum=0.9), loss='binary_crossentropy', metrics=['accuracy'])

In [21]:
# Define TensorBoard callback
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=[tensorboard_callback]  # Add TensorBoard callback
)

Epoch 1/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 581s 2s/step - accuracy: 0.5661 - loss: 0.7085 - val_accuracy: 0.7884 - val_loss: 0.5319
Epoch 2/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 676s 3s/step - accuracy: 0.6894 - loss: 0.5942 - val_accuracy: 0.8280 - val_loss: 0.4620
Epoch 4/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 575s 2s/step - accuracy: 0.7369 - loss: 0.5417 - val_accuracy: 0.8334 - val_loss: 0.4290
Epoch 6/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 668s 3s/step - accuracy: 0.7545 - loss: 0.5104 - val_accuracy: 0.8359 - val_loss: 0.4075
Epoch 8/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step 

In [22]:
# Evaluate and Test the Model
test_loss, test_acc = model.evaluate(validation_generator)
print('Test accuracy:', test_acc)


64/64 ━━━━━━━━━━━━━━━━━━━━ 114s 2s/step - accuracy: 0.8381 - loss: 0.3902
Test accuracy: 0.8329213857650757
